In [1]:
import requests

In [2]:
url = "https://cluster.elasticsearch.dataesr.ovh/bsso-publications/_search"

In [3]:
header= {"authorization": open("../../.env_es_bsso", "r").read().strip() }

### Sommaire

* [Publications](#publications)
    * [Général](#publi_general)
        * [Dynamique d'ouverture](#dynamique_ouverture)
        * [Voies d'ouverture](#voie_ouverture)
        * [Genre](#genre)
        * [Langue](#langue)
        * [Financement](#financement)
        * [COI](#coi)
    * [Discipline](#publi_disciplines)
        * [Dynamique d'ouverture par discipline](#dynamique_ouverture_disciplines)
        * [Voies d'ouverture par discipline](#voie_ouverture_disciplines)
    * [Editeurs](#publi_editeurs)
        * [Dynamique d'ouverture pour editeur](#dynamique_ouverture_editeurs)
        * [Voies d'ouverture par discipline](#voie_ouverture_editeurs)
        * [Politique d'ouverture par discipline](#politique_ouverture_editeurs)
        * [Poids des revues prédatrices](#predateur)
        * [Répartition par licence ouverte](#licence)
        * [APC](#apc)
     


# 1. Publications <a class="anchor" id="publications"></a>

## 1.1 Général <a class="anchor" id="publi_general"></a>

### 1.1.1 Dynamique d'ouverture (2 graphes, barres et lignes) <a class="anchor" id="dynamique_ouverture"></a>

In [65]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {
  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "publication_year"
      },
      "aggs": {
        "by_is_oa": {
          "terms": {
            "field": "oa_details." + year + ".is_oa"
          }
        }
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

{'by_publication_year': {'doc_count_error_upper_bound': 0,
  'sum_other_doc_count': 0,
  'buckets': [{'key': 2020.0,
    'doc_count': 54312,
    'by_is_oa': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 1, 'key_as_string': 'true', 'doc_count': 32272},
      {'key': 0, 'key_as_string': 'false', 'doc_count': 22040}]}},
   {'key': 2019.0,
    'doc_count': 45261,
    'by_is_oa': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 1, 'key_as_string': 'true', 'doc_count': 28785},
      {'key': 0, 'key_as_string': 'false', 'doc_count': 16476}]}},
   {'key': 2018.0,
    'doc_count': 44202,
    'by_is_oa': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 1, 'key_as_string': 'true', 'doc_count': 27589},
      {'key': 0, 'key_as_string': 'false', 'doc_count': 16613}]}},
   {'key': 2017.0,
    'doc_count': 17342,
    'by_is_oa': {'doc_count_error_upper_bound': 0,
     'sum_

### 1.1.2 Voies d'ouverture <a class="anchor" id="voie_ouverture"></a>

In [52]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {
  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "publication_year"
      },
      "aggs": {
        "by_oa_host_type": {
          "terms": {
            "field": "oa_details." + year + ".oa_host_type.keyword"
          }
        }
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

{'by_publication_year': {'doc_count_error_upper_bound': 0,
  'sum_other_doc_count': 0,
  'buckets': [{'key': 2020.0,
    'doc_count': 154,
    'by_oa_host_type': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 'publisher;repository', 'doc_count': 66},
      {'key': 'closed', 'doc_count': 62},
      {'key': 'repository', 'doc_count': 16},
      {'key': 'publisher', 'doc_count': 10}]}},
   {'key': 2019.0,
    'doc_count': 138,
    'by_oa_host_type': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 'publisher;repository', 'doc_count': 64},
      {'key': 'closed', 'doc_count': 45},
      {'key': 'repository', 'doc_count': 23},
      {'key': 'publisher', 'doc_count': 6}]}},
   {'key': 2018.0,
    'doc_count': 133,
    'by_oa_host_type': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 'publisher;repository', 'doc_count': 69},
      {'key': 'closed', 'doc_count': 38}

### 1.1.3 Genre de publications <a class="anchor" id="genre"></a>

In [17]:
year = "2021Q1" # latest
params = {
  "aggs": {
    "by_publication_genre": {
      "terms": {
        "field": "genre.keyword"
      },
      "aggs": {
        "by_oa_host_type": {
          "terms": {
            "field": "oa_details." + year + ".is_oa"
          }
        }
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

{'by_publication_genre': {'doc_count_error_upper_bound': 0,
  'sum_other_doc_count': 0,
  'buckets': [{'key': 'journal-article',
    'doc_count': 984,
    'by_oa_host_type': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 1, 'key_as_string': 'true', 'doc_count': 557},
      {'key': 0, 'key_as_string': 'false', 'doc_count': 425}]}},
   {'key': 'book-chapter',
    'doc_count': 13,
    'by_oa_host_type': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 0, 'key_as_string': 'false', 'doc_count': 11},
      {'key': 1, 'key_as_string': 'true', 'doc_count': 2}]}}]}}

### 1.1.4 Langue <a class="anchor" id="langue"></a>

In [20]:
year = "2021Q1" #latest
params = {
  "aggs": {
    "by_publication_genre": {
      "terms": {
        "field": "language.keyword"
      },
      "aggs": {
        "by_oa_host_type": {
          "terms": {
            "field": "oa_details." + year + ".is_oa"
          }
        }
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

{'by_publication_genre': {'doc_count_error_upper_bound': 0,
  'sum_other_doc_count': 0,
  'buckets': [{'key': 'en',
    'doc_count': 923,
    'by_oa_host_type': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 1, 'key_as_string': 'true', 'doc_count': 551},
      {'key': 0, 'key_as_string': 'false', 'doc_count': 370}]}},
   {'key': 'fr',
    'doc_count': 73,
    'by_oa_host_type': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 0, 'key_as_string': 'false', 'doc_count': 66},
      {'key': 1, 'key_as_string': 'true', 'doc_count': 7}]}},
   {'key': 'sp',
    'doc_count': 1,
    'by_oa_host_type': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 1, 'key_as_string': 'true', 'doc_count': 1}]}}]}}

### 1.1.6 Déclaration de financement <a class="anchor" id="financement"></a>

barres

In [24]:
year = "2021Q1" #latest
params = {
  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "publication_year"
      },
      "aggs": {
        "by_is_oa": {
          "terms": {
            "field": "oa_details." + year + ".is_oa"
          },
          "aggs": {
              "by_has_grant": {
                  "terms": {
                      "field": "has_grant"
                  }
              }
          }
        }
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

{'by_publication_year': {'doc_count_error_upper_bound': 0,
  'sum_other_doc_count': 0,
  'buckets': [{'key': 2020.0,
    'doc_count': 176,
    'by_is_oa': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 1,
       'key_as_string': 'true',
       'doc_count': 106,
       'by_has_grant': {'doc_count_error_upper_bound': 0,
        'sum_other_doc_count': 0,
        'buckets': [{'key': 0, 'key_as_string': 'false', 'doc_count': 88},
         {'key': 1, 'key_as_string': 'true', 'doc_count': 18}]}},
      {'key': 0,
       'key_as_string': 'false',
       'doc_count': 70,
       'by_has_grant': {'doc_count_error_upper_bound': 0,
        'sum_other_doc_count': 0,
        'buckets': [{'key': 0, 'key_as_string': 'false', 'doc_count': 62},
         {'key': 1, 'key_as_string': 'true', 'doc_count': 8}]}}]}},
   {'key': 2019.0,
    'doc_count': 129,
    'by_is_oa': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 1,
 

treemap (par agence)

In [28]:
year = "2021Q1" #latest
params = {
  "aggs": {
    "by_oa_host_type": {
      "terms": {
        "field": "oa_details." + year + ".oa_host_type.keyword"
      },
      "aggs": {
        "by_grant_agency": {
          "terms": {
            "field": "grants.agency.keyword"
          }
        }
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

{'by_oa_host_type': {'doc_count_error_upper_bound': 0,
  'sum_other_doc_count': 0,
  'buckets': [{'key': 'repository',
    'doc_count': 461,
    'by_grant_agency': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 85,
     'buckets': [{'key': 'NIAID NIH HHS', 'doc_count': 10},
      {'key': 'NIGMS NIH HHS', 'doc_count': 10},
      {'key': 'NCI NIH HHS', 'doc_count': 9},
      {'key': 'Wellcome Trust', 'doc_count': 9},
      {'key': 'Medical Research Council', 'doc_count': 8},
      {'key': 'NHLBI NIH HHS', 'doc_count': 8},
      {'key': 'European Research Council', 'doc_count': 5},
      {'key': 'NIDDK NIH HHS', 'doc_count': 5},
      {'key': 'Agence Nationale de la Recherche', 'doc_count': 4},
      {'key': 'Biotechnology and Biological Sciences Research Council',
       'doc_count': 4}]}},
   {'key': 'closed',
    'doc_count': 436,
    'by_grant_agency': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 30,
     'buckets': [{'key': 'CIHR', 'doc_count': 2},
 

### 1.1.7 COI <a class="anchor" id="coi"></a>

In [30]:
year = "2021Q1" #latest
params = {
  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "publication_year"
      },
      "aggs": {
        "by_has_coi": {
          "terms": {
            "field": "has_coi"
          }
        }
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

{'by_publication_year': {'doc_count_error_upper_bound': 0,
  'sum_other_doc_count': 0,
  'buckets': [{'key': 2020.0,
    'doc_count': 176,
    'by_has_coi': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 0, 'key_as_string': 'false', 'doc_count': 172},
      {'key': 1, 'key_as_string': 'true', 'doc_count': 4}]}},
   {'key': 2019.0,
    'doc_count': 129,
    'by_has_coi': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 0, 'key_as_string': 'false', 'doc_count': 127},
      {'key': 1, 'key_as_string': 'true', 'doc_count': 2}]}},
   {'key': 2016.0,
    'doc_count': 119,
    'by_has_coi': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 0, 'key_as_string': 'false', 'doc_count': 117},
      {'key': 1, 'key_as_string': 'true', 'doc_count': 2}]}},
   {'key': 2017.0,
    'doc_count': 118,
    'by_has_coi': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 

## 1.2 Disciplines <a class="anchor" id="publi_disciplines"></a>

### 1.2.1 Dynamique par discipline  <a class="anchor" id="dynamique_ouverture_disciplines"></a>

In [33]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {
  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "publication_year"
      },
      "aggs": {
        "by_is_oa": {
          "terms": {
            "field": "oa_details." + year + ".is_oa"
          },
          "aggs": {
            "by_discipline": {
              "terms": {
                "field": "bsso_fields.keyword"
              }
            }
          }
        }
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

{'by_publication_year': {'doc_count_error_upper_bound': 0,
  'sum_other_doc_count': 0,
  'buckets': [{'key': 2020.0,
    'doc_count': 176,
    'by_is_oa': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 1,
       'key_as_string': 'true',
       'doc_count': 106,
       'by_discipline': {'doc_count_error_upper_bound': 0,
        'sum_other_doc_count': 13,
        'buckets': [{'key': 'Clinical Sciences', 'doc_count': 30},
         {'key': 'Multidisciplinary', 'doc_count': 26},
         {'key': 'Microbiology', 'doc_count': 9},
         {'key': 'Medicinal and Biomolecular Chemistry', 'doc_count': 7},
         {'key': 'Oncology and Carcinogenesis', 'doc_count': 5},
         {'key': 'Public Health and Health Services', 'doc_count': 5},
         {'key': 'Biochemistry and Cell Biology', 'doc_count': 4},
         {'key': 'Biomedical Engineering', 'doc_count': 3},
         {'key': 'Cardiorespiratory Medicine and Haematology', 'doc_count': 3},
         {

### 1.2.2 Voie d'ouverture par discipline  <a class="anchor" id="voie_ouverture_disciplines"></a>

In [34]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {
  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "publication_year"
      },
      "aggs": {
        "by_is_oa": {
          "terms": {
            "field": "oa_details." + year + ".oa_host_type.keyword"
          },
          "aggs": {
            "by_discipline": {
              "terms": {
                "field": "bsso_fields.keyword"
              }
            }
          }
        }
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

{'by_publication_year': {'doc_count_error_upper_bound': 0,
  'sum_other_doc_count': 0,
  'buckets': [{'key': 2020.0,
    'doc_count': 176,
    'by_is_oa': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 'repository',
       'doc_count': 91,
       'by_discipline': {'doc_count_error_upper_bound': 0,
        'sum_other_doc_count': 12,
        'buckets': [{'key': 'Multidisciplinary', 'doc_count': 25},
         {'key': 'Clinical Sciences', 'doc_count': 24},
         {'key': 'Microbiology', 'doc_count': 9},
         {'key': 'Medicinal and Biomolecular Chemistry', 'doc_count': 5},
         {'key': 'Oncology and Carcinogenesis', 'doc_count': 4},
         {'key': 'Public Health and Health Services', 'doc_count': 4},
         {'key': 'Biochemistry and Cell Biology', 'doc_count': 3},
         {'key': 'Biomedical Engineering', 'doc_count': 2},
         {'key': 'Genetics', 'doc_count': 2},
         {'key': 'Pharmacology and Pharmaceutical Sciences',
     

## 1.3 Editeurs <a class="anchor" id="publi_editeurs"></a>

### 1.3.1 Dynamique pour editeurs  <a class="anchor" id="dynamique_ouverture_editeurs"></a>

In [59]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {
  "query": {
    "bool": {
      "filter": [
        { "term": { "oa_details." + year + ".oa_host_type": "publisher"   }} # ATTENTION pas de .keyword ici pour avoir publisher et publisher;repository
       # ,{ "term": { "publisher.keyword": "Elsevier BV"   }} ## pour un éditeur donné
      ]
    }
  },
  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "publication_year"
      },
      "aggs": {
        "by_is_oa": {
          "terms": {
            "field": "oa_details." + year + ".is_oa"
          }
        }
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

{'by_publication_year': {'doc_count_error_upper_bound': 0,
  'sum_other_doc_count': 0,
  'buckets': [{'key': 2018.0,
    'doc_count': 81,
    'by_is_oa': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 1, 'key_as_string': 'true', 'doc_count': 81}]}},
   {'key': 2020.0,
    'doc_count': 76,
    'by_is_oa': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 1, 'key_as_string': 'true', 'doc_count': 76}]}},
   {'key': 2019.0,
    'doc_count': 70,
    'by_is_oa': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 1, 'key_as_string': 'true', 'doc_count': 70}]}},
   {'key': 2017.0,
    'doc_count': 65,
    'by_is_oa': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 1, 'key_as_string': 'true', 'doc_count': 65}]}},
   {'key': 2016.0,
    'doc_count': 54,
    'by_is_oa': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count':

### 1.3.2 Voie d'ouverture pour editeurs <a class="anchor" id="voie_ouverture_editeurs"></a>


In [61]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {
  "query": {
    "bool": {
      "filter": [
        { "term": { "oa_details." + year + ".oa_host_type": "publisher"   }} # ATTENTION pas de .keyword ici pour avoir publisher et publisher;repository
       # ,{ "term": { "publisher.keyword": "Elsevier BV"   }} ## pour un éditeur donné
      ]
    }
  },
  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "publication_year"
      },
      "aggs": {
        "by_is_oa_host_type": {
          "terms": {
            "field": "oa_details." + year + ".oa_colors.keyword"
          }
        }
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

{'by_publication_year': {'doc_count_error_upper_bound': 0,
  'sum_other_doc_count': 0,
  'buckets': [{'key': 2018.0,
    'doc_count': 81,
    'by_is_oa_host_type': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 'green', 'doc_count': 69},
      {'key': 'gold', 'doc_count': 49},
      {'key': 'bronze', 'doc_count': 23},
      {'key': 'hybrid', 'doc_count': 9}]}},
   {'key': 2020.0,
    'doc_count': 76,
    'by_is_oa_host_type': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 'green', 'doc_count': 66},
      {'key': 'gold', 'doc_count': 56},
      {'key': 'hybrid', 'doc_count': 15},
      {'key': 'bronze', 'doc_count': 5}]}},
   {'key': 2019.0,
    'doc_count': 70,
    'by_is_oa_host_type': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 'green', 'doc_count': 64},
      {'key': 'gold', 'doc_count': 46},
      {'key': 'bronze', 'doc_count': 19},
      {'key': 'h

### 1.3.3 Politique d'ouverture par editeur <a class="anchor" id="politique_ouverture_editeurs"></a>

In [66]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {
  "query": {
    "bool": {
      "filter": [
     #   { "term": { "oa_details." + year + ".oa_host_type": "publisher"   }} # ATTENTION pas de .keyword ici pour avoir publisher et publisher;repository
       { "term": { "publication_year": 2020   }} ## pour une publication year donnée
      ]
    }
  },
  "aggs": {
    "by_publisher": {
      "terms": {
        "field": "publisher.keyword"
      },
      "aggs": {
        "by_is_oa_host_type": {
          "terms": {
            "field": "oa_details." + year + ".oa_colors.keyword"
          }
        }
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

{'by_publisher': {'doc_count_error_upper_bound': 0,
  'sum_other_doc_count': 11051,
  'buckets': [{'key': 'Elsevier BV',
    'doc_count': 15192,
    'by_is_oa_host_type': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 'closed', 'doc_count': 9415},
      {'key': 'green', 'doc_count': 4330},
      {'key': 'gold', 'doc_count': 1440},
      {'key': 'hybrid', 'doc_count': 1398},
      {'key': 'bronze', 'doc_count': 1097}]}},
   {'key': 'Springer Science and Business Media LLC',
    'doc_count': 8741,
    'by_is_oa_host_type': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 'green', 'doc_count': 5817},
      {'key': 'gold', 'doc_count': 4142},
      {'key': 'closed', 'doc_count': 2547},
      {'key': 'hybrid', 'doc_count': 812},
      {'key': 'bronze', 'doc_count': 667}]}},
   {'key': 'Wiley',
    'doc_count': 5734,
    'by_is_oa_host_type': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,

### 1.3.4 Poids des revues prédatrices <a class="anchor" id="predateur"></a>

In [68]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {
  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "publication_year"
      },
      "aggs": {
        "by_predatory": {
          "terms": {
            "field": "predatory_journal"
          }
        }
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

{'by_publication_year': {'doc_count_error_upper_bound': 0,
  'sum_other_doc_count': 0,
  'buckets': [{'key': 2020.0,
    'doc_count': 54312,
    'by_predatory': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 0, 'key_as_string': 'false', 'doc_count': 54140},
      {'key': 1, 'key_as_string': 'true', 'doc_count': 172}]}},
   {'key': 2019.0,
    'doc_count': 45261,
    'by_predatory': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 0, 'key_as_string': 'false', 'doc_count': 45081},
      {'key': 1, 'key_as_string': 'true', 'doc_count': 180}]}},
   {'key': 2018.0,
    'doc_count': 44202,
    'by_predatory': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 0, 'key_as_string': 'false', 'doc_count': 43851},
      {'key': 1, 'key_as_string': 'true', 'doc_count': 351}]}},
   {'key': 2017.0,
    'doc_count': 17342,
    'by_predatory': {'doc_count_error_upper_bound': 0,


### 1.3.5 Licences <a class="anchor" id="licence"></a>


global

In [76]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {
  "query": {
    "bool": {
      "filter": [
     #   { "term": { "oa_details." + year + ".oa_host_type": "publisher"   }} # ATTENTION pas de .keyword ici pour avoir publisher et publisher;repository
       { "term": { "publication_year": 2020   }} ## pour une publication year donnée
    #  ,{ "term": { "publisher.keyword": "Elsevier BV"   }} ## pour un éditeur donné
      ]
    }
  },
  "aggs": {
    "by_licence": {
      "terms": {
        "field": "oa_details." + year + ".licence_publisher.keyword"
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

{'by_licence': {'doc_count_error_upper_bound': 0,
  'sum_other_doc_count': 0,
  'buckets': [{'key': 'cc-by-nc-nd', 'doc_count': 2287},
   {'key': 'no license', 'doc_count': 1103},
   {'key': 'cc-by', 'doc_count': 537},
   {'key': 'implied-oa', 'doc_count': 7},
   {'key': 'cc-by-nc-sa', 'doc_count': 1}]}}

par éditeur

In [70]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {
  "query": {
    "bool": {
      "filter": [
     #   { "term": { "oa_details." + year + ".oa_host_type": "publisher"   }} # ATTENTION pas de .keyword ici pour avoir publisher et publisher;repository
       { "term": { "publication_year": 2020   }} ## pour une publication year donnée
      ]
    }
  },
  "aggs": {
    "by_publisher": {
      "terms": {
        "field": "publisher.keyword"
      },
      "aggs": {
        "by_licence": {
          "terms": {
            "field": "oa_details." + year + ".licence_publisher.keyword"
          }
        }
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

{'by_publisher': {'doc_count_error_upper_bound': 0,
  'sum_other_doc_count': 11051,
  'buckets': [{'key': 'Elsevier BV',
    'doc_count': 15192,
    'by_licence': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 'cc-by-nc-nd', 'doc_count': 2287},
      {'key': 'no license', 'doc_count': 1103},
      {'key': 'cc-by', 'doc_count': 537},
      {'key': 'implied-oa', 'doc_count': 7},
      {'key': 'cc-by-nc-sa', 'doc_count': 1}]}},
   {'key': 'Springer Science and Business Media LLC',
    'doc_count': 8741,
    'by_licence': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 'cc-by', 'doc_count': 4803},
      {'key': 'no license', 'doc_count': 671},
      {'key': 'cc-by-nc', 'doc_count': 133},
      {'key': 'cc-by-nc-nd', 'doc_count': 8},
      {'key': 'publisher-specific', 'doc_count': 4},
      {'key': 'cc-by-nc-sa', 'doc_count': 2}]}},
   {'key': 'Wiley',
    'doc_count': 5734,
    'by_licence': {'doc_count

### 1.3.6 APC <a class="anchor" id="apc"></a>


somme des apc par an

In [74]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {

  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "publication_year"
      },
      "aggs": {
        "apc": {
          "sum": {
            "field": "amount_apc_EUR"
          }
        }
      }
    }
  }
}

requests.get(url, json=params, headers=header).json()['aggregations']

{'by_publication_year': {'doc_count_error_upper_bound': 0,
  'sum_other_doc_count': 0,
  'buckets': [{'key': 2020.0,
    'doc_count': 54312,
    'apc': {'value': 46918544.644563675}},
   {'key': 2019.0, 'doc_count': 45261, 'apc': {'value': 62285418.884376526}},
   {'key': 2018.0, 'doc_count': 44202, 'apc': {'value': 69641852.66469955}},
   {'key': 2017.0, 'doc_count': 17342, 'apc': {'value': 27692346.85294342}},
   {'key': 2021.0, 'doc_count': 10279, 'apc': {'value': 6185461.524078369}},
   {'key': 2016.0, 'doc_count': 98, 'apc': {'value': 106869.97399902344}},
   {'key': 2015.0, 'doc_count': 4, 'apc': {'value': 3415.805908203125}},
   {'key': 2013.0, 'doc_count': 1, 'apc': {'value': 0.0}},
   {'key': 2014.0, 'doc_count': 1, 'apc': {'value': 0.0}}]}}

distribution des tarifs

In [78]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {
    
"query": {
    "bool": {
      "filter": [
  #  { "term": { "publisher.keyword": "Elsevier BV"   }} ## pour un éditeur donné
      ]
    }
  },

  "aggs": {
        "tarif": {
          "histogram": {
            "field": "amount_apc_EUR",
            "interval": 250
          }
        }
      }
    }


requests.get(url, json=params, headers=header).json()['aggregations']

{'tarif': {'buckets': [{'key': 0.0, 'doc_count': 255},
   {'key': 250.0, 'doc_count': 830},
   {'key': 500.0, 'doc_count': 1681},
   {'key': 750.0, 'doc_count': 1604},
   {'key': 1000.0, 'doc_count': 2908},
   {'key': 1250.0, 'doc_count': 7970},
   {'key': 1500.0, 'doc_count': 14557},
   {'key': 1750.0, 'doc_count': 10629},
   {'key': 2000.0, 'doc_count': 10031},
   {'key': 2250.0, 'doc_count': 7929},
   {'key': 2500.0, 'doc_count': 8174},
   {'key': 2750.0, 'doc_count': 6582},
   {'key': 3000.0, 'doc_count': 5397},
   {'key': 3250.0, 'doc_count': 3318},
   {'key': 3500.0, 'doc_count': 2750},
   {'key': 3750.0, 'doc_count': 1836},
   {'key': 4000.0, 'doc_count': 1894},
   {'key': 4250.0, 'doc_count': 1695},
   {'key': 4500.0, 'doc_count': 784},
   {'key': 4750.0, 'doc_count': 593},
   {'key': 5000.0, 'doc_count': 420},
   {'key': 5250.0, 'doc_count': 313},
   {'key': 5500.0, 'doc_count': 63},
   {'key': 5750.0, 'doc_count': 4},
   {'key': 6000.0, 'doc_count': 2},
   {'key': 6250.0, 'do

violin : 1. one request for density

In [81]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {
    
"query": {
    "bool": {
      "filter": [
  #  { "term": { "publisher.keyword": "Elsevier BV"   }} ## pour un éditeur donné
      ]
    }
  },
  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "publication_year"
      },
      "aggs": {
            "tarif": {
              "histogram": {
                "field": "amount_apc_EUR",
                "interval": 250
              }
            }
      }
      }
    }
}


requests.get(url, json=params, headers=header).json()['aggregations']

{'by_publication_year': {'doc_count_error_upper_bound': 0,
  'sum_other_doc_count': 0,
  'buckets': [{'key': 2020.0,
    'doc_count': 54312,
    'tarif': {'buckets': [{'key': 0.0, 'doc_count': 42},
      {'key': 250.0, 'doc_count': 159},
      {'key': 500.0, 'doc_count': 364},
      {'key': 750.0, 'doc_count': 414},
      {'key': 1000.0, 'doc_count': 658},
      {'key': 1250.0, 'doc_count': 1694},
      {'key': 1500.0, 'doc_count': 5061},
      {'key': 1750.0, 'doc_count': 3250},
      {'key': 2000.0, 'doc_count': 2610},
      {'key': 2250.0, 'doc_count': 1365},
      {'key': 2500.0, 'doc_count': 1126},
      {'key': 2750.0, 'doc_count': 1114},
      {'key': 3000.0, 'doc_count': 704},
      {'key': 3250.0, 'doc_count': 674},
      {'key': 3500.0, 'doc_count': 539},
      {'key': 3750.0, 'doc_count': 500},
      {'key': 4000.0, 'doc_count': 78},
      {'key': 4250.0, 'doc_count': 897},
      {'key': 4500.0, 'doc_count': 145},
      {'key': 4750.0, 'doc_count': 52},
      {'key': 5000.0,

violin : second request for percentiles

In [82]:
year = "2021Q1" # et aussi tous les "millésimes" 2018, 2019, 2020
params = {
    
"query": {
    "bool": {
      "filter": [
  #  { "term": { "publisher.keyword": "Elsevier BV"   }} ## pour un éditeur donné
      ]
    }
  },
  "aggs": {
    "by_publication_year": {
      "terms": {
        "field": "publication_year"
      },
      "aggs": {
            "tarif_percentiles": {
              "percentiles": {
                "field": "amount_apc_EUR"
              }
            }
      }
      }
    }
}


requests.get(url, json=params, headers=header).json()['aggregations']

{'by_publication_year': {'doc_count_error_upper_bound': 0,
  'sum_other_doc_count': 0,
  'buckets': [{'key': 2020.0,
    'doc_count': 54312,
    'tarif_percentiles': {'values': {'1.0': 512.4208392878942,
      '5.0': 1038.800048828125,
      '25.0': 1622.968181356215,
      '50.0': 1904.6500236430923,
      '75.0': 2659.656349539428,
      '95.0': 4393.225912475586,
      '99.0': 4537.6806640625}}},
   {'key': 2019.0,
    'doc_count': 45261,
    'tarif_percentiles': {'values': {'1.0': 506.631988840739,
      '5.0': 1016.7063623700825,
      '25.0': 1664.1996483803778,
      '50.0': 2155.4659239261628,
      '75.0': 2799.253517995112,
      '95.0': 4139.791015625,
      '99.0': 4938.76220703125}}},
   {'key': 2018.0,
    'doc_count': 44202,
    'tarif_percentiles': {'values': {'1.0': 403.81253973992324,
      '5.0': 987.8507884149966,
      '25.0': 1659.2239602981135,
      '50.0': 2344.4661603947875,
      '75.0': 2988.388681034409,
      '95.0': 4099.7890625,
      '99.0': 5145.874951